In [58]:
import pandas as pd
import numpy as np
import requests
import os
import json
import datetime
import time

In [3]:
df = pd.read_csv("Twitter-Usernames.csv")
df.tail()

,Journalist,Publisher,first name,last name,twitter_username_1,twitter_username_2,twitter_username_3,submitted username,bio,twitter id
3590,"Harris, Gardiner",New York Times,gardiner,harris,gardinerharris,gharris,gardinerh,gardinerh,News Producer,258870975
3591,"Harris, Shane",Wall Street Journal/Dow Jones,shane,harris,shaneharris,sharris,shaneh,shaneh,"A man, a plan, a canal, Panama.",14372842
3592,"Hart, Dan",Bloomberg News,dan,hart,danhart,dhart,danh,danh,"Co-founder & CTO, http://t.co/uzcNWewtHY",4594
3593,"Harte, Julia",Thomson Reuters,julia,harte,juliaharte,jharte,juliah,juliah,NaN,133236174
3594,"Hartson, Merrill",Associated Press,merrill,hartson,merrillhartson,mhartson,merrillh,merrillh,Produce various Local TV shows. See LinkedIn,22254372


In [4]:
re_df = pd.read_csv('/home/sagemaker-user/BeltwayJournalists/1123_raw_post_measures.csv')
re_df.head()

,Id,modularity_class,user_id,user_name,Organization,Catergory_1,Catergory_2,Position,Gender,Race,...,weighted indegree,weighted outdegree,Weighted Degree,closnesscentrality,betweenesscentrality,clustering,eigencentrality,Authority,Hub,Notes
0,greta,0,16031927.0,Greta Van Susteren,MSNBC,Television Network News and Public Affairs,Radio_and_Television,"Anchor, For the Record",F,White,...,117,144,261,0.365522,16740.756400,0.148033,0.099662,0.014454,0.033746,"Senate lists says ""Fox News."""
1,ZekeJMiller,0,21316253.0,Zeke Miller,Associated Press,Wire Services,Senate_Press_Galleries,White House Correspondent,M,White,...,854,88,942,0.343639,14632.497050,0.099029,0.729228,0.128160,0.026938,-
2,sarahkliff,0,19734832.0,Sarah Kliff,Vox Media,Digital,Senate_Periodical_Galleries,NaN,F,White,...,78,28,106,0.312724,1769.316764,0.165007,0.037871,0.012408,0.001922,-
3,BCAppelbaum,0,87271106.0,Binyamin Appelbaum,New York Times,Newspapers,Senate_Press_Galleries,NaN,M,White,...,165,60,225,0.330305,6794.760176,0.103918,0.180851,0.033629,0.012413,"Lists ""Facebook: Binyamin.Appelbaum"""
4,jeneps,0,45399148.0,Jennifer Epstein,Bloomberg News,News Magazine,Senate_Press_Galleries,Political Reporter,F,White,...,385,174,559,0.372306,19469.900720,0.138686,0.460284,0.082985,0.037160,-


In [35]:
def auth():
    consumer_key = "B3yBDzWAjgI9x3gHIsiEY2UMj"
    consumer_secret = "5J7pd0aNqnQViRQgRO57WNa8ujLaR4coHDecVzbSVFhbNileYc"
    data = {
      'grant_type': 'client_credentials'
    }

    response = requests.post('https://api.twitter.com/oauth2/token', data=data, auth=(consumer_key, consumer_secret))
    return response.json()['access_token']

def create_url(username):
    query = "from:{}".format(username)

    tweet_fields = "tweet.fields=author_id"
    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}".format(
        query, tweet_fields
    )
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, i):
    response = requests.request("GET", url, headers=headers)
    code = response.status_code
    if code != 200:
        if code == 429:
            print(datetime.datetime.now())
            print(i)
            return code
        else:
            raise Exception(response.status_code, response.text)
    else:
        return response.json()



In [53]:
def get_tweets(username, i):
    bearer_token = auth()
    url = create_url(username)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers, i)

    if json_response == 429:
        return 429
    
    elif len(json_response) <= 1:
        return pd.DataFrame()
    
    else:
        tweet_data = json_response['data']

        tweet_id = []
        tweet_text = []

        for i in range(0,len(tweet_data)):
            this_tweet = tweet_data[i]
            tweet_id.append(this_tweet['id'])
            tweet_text.append(this_tweet['text'])
            
        return pd.DataFrame({'username': username,
                      'tweet_id': tweet_id,
                      'tweet_text': tweet_text})


In [51]:
all_journal = np.array(list(df['submitted username']) + list(re_df['Id']))
all_journal = np.unique(all_journal)
print(len(all_journal))

4068


In [ ]:
#dfs = pd.DataFrame()

for i, user in enumerate(all_journal[1000:]):
    
    resp = get_tweets(user, i)
    
    try:
        dfs = pd.concat([dfs, resp])
    except:
        if resp == 429:
            dfs = pd.concat([dfs, pd.DataFrame()])
            print('Waiting')
            time.sleep(60*17)

2020-12-20 22:27:43.386550
0
Waiting
2020-12-20 22:47:27.554514
451
Waiting
2020-12-20 23:07:05.096213
902
Waiting
2020-12-20 23:26:44.516652
1353
Waiting


In [ ]:
print(dfs.shape)
dfs.to_csv("User_tweets.csv", index=False)
dfs.head()